In [1]:
# Importar librerías.
import csv
import requests
from bs4 import BeautifulSoup
import json

Función que realiza consultas a la [web](https://codigo-postal.co/argentina/) para extraer las variables __provincia__ y __localidad__.
El código proporcionado en la función llamada __extraer_localities_base()__ se encarga de extraer datos de localidades y provincias del sitio [web](https://codigo-postal.co/argentina/)  y guardarlos en un archivo CSV(_localities_new.csv_). A continuación, se comentan las líneas de código para comprender su significado:

In [ ]:
def extraer_localities_base():
    
    """En estas líneas, se abre un archivo JSON llamado 'statesCode.json' y se carga su contenido en la variable provincias_list.
    El archivo conteniene nombres de provincias."""
    with open('data/statesCode.json', 'r') as archivo:
        provincias_list = list(dict(json.load(archivo)).keys())

    """Aquí se crean listas vacías llamadas localidades y provincias, y se inicializa una sesión de solicitud HTTP usando requests.Session().
    Luego, se envía una solicitud GET a la URL 'https://codigo-postal.co/argentina/' y se guarda la respuesta en response1."""
    localidades = []
    provincias = []
    session = requests.Session()
    response1 = session.get('https://codigo-postal.co/argentina/')

    """En este bucle for, se itera sobre cada provincia en provincias_list.
    Se realiza una manipulación de cadenas en la variable pro para formatear la provincia en un formato adecuado para la URL.
    Luego, se envía una solicitud GET a una URL específica para cada provincia y se guarda la respuesta en response2.
    Si el código de estado de la respuesta es 200 (indicando una respuesta exitosa),
    se guarda el contenido de la respuesta en content y se crea un objeto BeautifulSoup llamado soup para analizar el contenido HTML."""
    for p in provincias_list:
        
        pro = p.lower().replace(" ","-").replace("ñ","n")
        response2 = session.get(f'https://codigo-postal.co/argentina/{pro}')
        if response2.status_code == 200:
            content = response2.content
            soup = BeautifulSoup(content, "html.parser")
    
        """En estas líneas, se busca en el contenido HTML de la página ciertos elementos para obtener una lista de localidades.
        Se utiliza soup.find() para encontrar un elemento con la clase "container content_container", luego se busca un elemento "ul"
        y finalmente se encuentran todos los elementos "li" dentro de él. Se extrae el texto de los elementos "a" dentro de cada "li"
        y se guarda en la lista localidad. Luego, se extiende la lista localidades con los elementos de localidad.
        Además, se crea una lista de provincias llamada provincia donde cada elemento es el nombre de la provincia actual (p)
        repetido tantas veces como localidades haya en la lista localidad.
        Finalmente, se extiende la lista provincias con los elementos de provincia."""
        lista = soup.find(class_="container content_container").find("ul").find_all("li")
        localidad = [i.find("a").get_text() for i in lista]
        localidades.extend(localidad)  
        provincia = [p] * len(localidad)
        provincias.extend(provincia)

    """En estas líneas, se abre un nuevo archivo CSV llamado 'localities_new.csv' en modo de escritura. Se definen los nombres de campo ('provincia' y 'localidad')
    y se crea un escritor de CSV (csv.DictWriter) usando el archivo y los nombres de campo. Luego, se itera sobre la lista combinada de provincias
    y localidades utilizando zip() para obtener pares de valores correspondientes.
    En cada iteración, se escribe una fila en el archivo CSV con los valores de 'provincia' y 'localidad' correspondientes."""
    with open('data/localities_new.csv', 'w', newline='') as file:
        fieldnames = ['provincia', 'localidad']
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        
        for k in list(zip(provincias,localidades)):
            
            writer.writerow({'provincia': k[0], 'localidad': k[1]})

    """Finalmente, la función no devuelve ningún valor explícito, ya que solo se encarga de realizar la extracción
    y guardado de los datos en el archivo CSV."""
    return

### Todas las funciones de extracción realizan consultas por provincia, para un mejor analisis funcional. 

Función que realiza consultas a la [web](https://codigo-postal.co/argentina/) para extraer la variable __cpa__ de cada localidad con menos de 500 habitantes. Con más de 500 habitantes devuelve el valor _Buscar CPA_. El código proporcionado en la función llamada extraer_localities(provincias) se encarga de extraer datos de localidades y códigos postales del sitio [web](https://codigo-postal.co/argentina/) y guardarlos en un nuevo archivo CSV específico para cada provincia. A continuación, se comentan las líneas de código para comprender su significado:

In [ ]:
def extraer_localities(provincias):
    
    """Aquí se abre un archivo CSV llamado "localities_new.csv", generado por la funcion anterior, en modo de lectura. Se lee el contenido del archivo utilizando csv.reader
    y se guarda en la variable csv_readers. Luego, se itera sobre los elementos de csv_readers
    y se verifica si la primera columna coincide con la provincia proporcionada (provincias). Si hay una coincidencia, se agrega el elemento a la lista csv_reader."""
    with open("data/localities_new.csv", 'r') as file:
        csv_readers = csv.reader(file)
        csv_reader = []
        for i in list(csv_readers):
            if i[0] == provincias:
                csv_reader.append(i)

        """Aquí se crea una sesión de solicitud HTTP utilizando requests.Session(). Luego, se realiza una solicitud GET a una URL específica para la provincia proporcionada.
        Después, se itera sobre los elementos de csv_reader. Para cada elemento, se inicializan las listas provincia, localidades
        y CPA_localidad. Se forma una cadena localidad concatenando la provincia y la localidad actual del elemento de csv_reader en un formato adecuado para la URL."""
        session = requests.Session()
        cpa_csv = []
        response1 = session.get(f'https://codigo-postal.co/argentina/{provincias}')
        for row in list(csv_reader):
            provincia = []
            localidades = []
            CPA_localidad = []
            localidad = f"{provincias}/{row[1]}".lower().replace(" ","-").replace("ñ","n")
            
            """Aquí se intenta construir la URL para la solicitud HTTP.
            Se verifica si la localidad no está en la columna 1 de cpa_csv
            y se construye la URL. En cualquier caso, se asegura de que la variable url tenga el valor correcto para la solicitud."""
            try:
                if row[1] not in list(zip(*cpa_csv))[1]:
                    url = f"https://codigo-postal.co/argentina/{localidad}"
                else:
                    continue
            except:
                url = f"https://codigo-postal.co/argentina/{localidad}"
            
            """Aquí se realiza una solicitud GET a la URL construida anteriormente utilizando la sesión HTTP. Si la respuesta tiene un código de estado 200 (éxito),
            se obtiene el contenido de la respuesta y se analiza utilizando BeautifulSoup para poder extraer datos de él.
            Si ocurre alguna excepción al realizar la solicitud o al analizar el contenido, se pasa a la siguiente iteración del bucle."""
            try:
                response2 = session.get(url)
                if response2.status_code == 200:
                    content = response2.content
                    soup = BeautifulSoup(content, "html.parser")
            except:
                continue
            
            """Aquí se intenta encontrar una tabla en el contenido analizado. Si se encuentra, se itera sobre las filas de la tabla
            y se extraen los valores de las columnas correspondientes a la provincia, localidad y CPA. Estos valores se agregan a las listas provincia,
            localidades y CPA_localidad. Si no se puede encontrar la tabla o extraer los valores, se agrega el valor de la provincia
            y la localidad del elemento actual de csv_reader a las listas correspondientes y se establece el valor "Buscar CPA" para el CPA."""
            try:
                table = soup.find("table").find("tbody").find_all("tr")
                # Extraer la variable "provincia".
                provincia.extend([i.find_all("td")[0].get_text() for i in table])
                # Extraer la variable localidad.
                localidades.extend([i.find_all("td")[1].get_text() for i in table])
                # Extraer la variable CPA.
                CPA_localidad.extend([i.find_all("td")[3].get_text() for i in table])
            except:
                provincia.append(row[0])
                localidades.append(row[1])
                CPA_localidad.append("Buscar CPA")
            
            """Aquí se crea una lista de tuplas combinando los valores de provincia, localidades
            y CPA_localidad. Luego, se convierte la lista en un conjunto para eliminar duplicados
            y finalmente se convierte nuevamente en una lista."""
            for i in list(zip(provincia,localidades,CPA_localidad)):
                cpa_csv.append(i)
            cpa_csv = list(set(cpa_csv))
    
    """Aquí se abre un nuevo archivo CSV con un nombre específico basado en la provincia proporcionada.
    Se especifican los nombres de campo para el encabezado del archivo CSV.
    Luego, se utiliza csv.DictWriter para escribir las filas en el archivo CSV utilizando los valores de las tuplas en cpa_csv."""
    with open(f'data/localities_{provincias}.csv', 'w', newline='') as file:
        fieldnames = ['localidad', 'cpa_localidad','provincia']
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        for k in cpa_csv:
            writer.writerow({'localidad': k[1], 'cpa_localidad': k[2],'provincia': k[0]})

    """En resumen, la función extraer_localities(provincias) lee el archivo "localities_new.csv" para obtener una lista de localidades
    y luego realiza solicitudes HTTP para obtener los códigos postales de esas localidades desde un sitio web. Luego, guarda los datos
    obtenidos en un nuevo archivo CSV específico para la provincia proporcionada."""
    return

Función que realiza consultas a la [web](https://codigo-postal.co/argentina/) para extraer la variable __calles/avenidas/rutas__. El código proporcionado en la función llamada extraer_streets(pro) se encarga de extraer datos de calles asociadas a localidades en una provincia específica desde un sitio web y guardarlos en un nuevo archivo CSV. A continuación, se comentan las líneas de código para comprender su significado:

In [57]:
def extraer_streets(pro):

    """Aquí se abre un archivo CSV específico, creado anteriormente, para la provincia proporcionada, llamado "localities_{pro}.csv", en modo de lectura.
    Se lee el contenido del archivo utilizando csv.reader y se itera sobre los elementos.
    Si la segunda columna del elemento es igual a "Buscar CPA", se agrega el valor de la primera columna (localidad) a la lista localidades."""
    localidades = []
    with open(f"data/localities_{pro}.csv", 'r') as file:
        csv_reader = csv.reader(file)
        for i in csv_reader:
            if i[1] == "Buscar CPA":
                localidades.append(i[0])

    """Aquí se crea una sesión de solicitud HTTP utilizando requests.Session(). Luego, se realiza una solicitud GET a una URL específica para la provincia proporcionada
    y se guarda la respuesta en response1. Se inicializan las listas street y localidad para almacenar los datos de las calles y las localidades, respectivamente."""
    session = requests.Session()
    response1 = session.get(f'https://codigo-postal.co/argentina/{pro}')
    street = []
    localidad = []

    """Aquí se itera sobre cada localidad en la lista localidades. Se construye una cadena loc a partir de la localidad actual,
    en un formato adecuado para la URL. Luego, se realiza una solicitud GET a una URL específica para la provincia y localidad actual,
    y se guarda la respuesta en response2. Si la respuesta tiene un código de estado 200 (éxito), se obtiene el contenido de la respuesta
    y se analiza utilizando BeautifulSoup para poder extraer datos de él. Se encuentra la lista de elementos HTML que contienen las calles
    y se extraen los nombres de las calles. Estos nombres se agregan a la lista street, y se crea una lista localidads con la localidad actual repetida para cada calle.
    Luego, se agregan las calles y las localidades a las listas street y localidad, respectivamente."""
    for p in localidades:
        loc = p.lower().replace(" ","-").replace("ñ","n")
        response2 = session.get(f'https://codigo-postal.co/argentina/{pro}/calles-de-{loc}')
        if response2.status_code == 200:
            content = response2.content
            soup = BeautifulSoup(content, "html.parser")
        lista = soup.find(class_="three_columns").find_all("li")
        streets = [i.find("a").get_text() for i in lista]
        street.extend(streets)
        localidads = [p] * len(streets)
        localidad.extend(localidads)
    
    """Aquí se abre un nuevo archivo CSV con un nombre específico basado en la provincia proporcionada,
    llamado "streets_{pro}.csv", en modo de escritura. Se especifican los nombres de campo para el encabezado del archivo CSV.
    Luego, se utiliza csv.DictWriter para escribir las filas en el archivo CSV utilizando los valores de las listas localidad
    y street en combinación. Cada fila representa una localidad y una calle asociada."""
    with open(f'data/streets_{pro}.csv', 'w', newline='') as file:
        fieldnames = ['localidad','street']
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        for k in list(zip(localidad,street)):
            writer.writerow({'localidad': k[0],'street':k[1]})

    """En resumen, la función extraer_streets(pro) lee un archivo CSV que contiene localidades asociadas a una provincia específica.
    Luego, realiza solicitudes HTTP para obtener las calles asociadas
    a cada localidad desde un sitio web y guarda los datos en un nuevo archivo CSV específico para la provincia proporcionada."""
    return

Función que realiza consultas a la [web](https://codigo-postal.co/argentina/) para extraer las variables __isOdd__, __from__, __until__ y __zip__ de cada calle/avenida/ruta. El código proporcionado en la función llamada extraer_numbers(pro) que se encarga de extraer datos de cpa de calles en una provincia específica desde un sitio [web](https://codigo-postal.co/argentina/) y guardarlos en un nuevo archivo CSV. A continuación, se comentan las líneas de código para comprender su significado:

In [6]:
def extraer_numbers(pro):

    """Aquí se abre un archivo CSV específico para la provincia proporcionada, llamado "streets_{pro}.csv",
    en modo de lectura. Se lee el contenido del archivo utilizando csv.reader y se itera sobre los elementos.
    Se agrega el valor de la primera columna (localidad) a la lista localidad,
    y se agrega el valor de la segunda columna (calle) a la lista calles."""
    calles = []
    localidad = []
    with open(f"data/streets_{pro}.csv", 'r') as file:
        csv_reader = csv.reader(file)
        for i in csv_reader:
            localidad.append(i[0])
            calles.append(i[1])

    """Aquí se crea una sesión de solicitud HTTP utilizando requests.Session(). Luego, se realiza una solicitud GET a una URL específica
    para la provincia proporcionada y se guarda la respuesta en response0.
    Se inicializan las listas street, isOdd, froms, until, zips y localidads para almacenar los datos de las calles."""
    session = requests.Session()
    response0 = session.get(f'https://codigo-postal.co/argentina/{pro}')
    street = []
    isOdd = []
    froms = []
    until = []
    zips = []
    localidads = []

    """A continuación, se itera sobre cada localidad en la lista localidad. Se formatea el nombre de la localidad
    para que coincida con la URL del sitio web. Luego, se realiza una solicitud GET a una URL específica para la provincia
    y la localidad. Si la respuesta es exitosa (código de estado 200),
    se extraen los datos de la tabla HTML que contiene la información de las calles."""
    for lo in localidad:
        loc = lo.lower().replace(" ","-").replace("ñ","n")
        response1 = session.get(f'https://codigo-postal.co/argentina/{pro}/{loc}')
        for c in calles:

            """Dentro del bucle for c in calles, se procesa cada calle de la lista calles. Se realiza una serie de transformaciones
            en el nombre de la calle para que coincida con la URL del sitio web. Luego, se realiza una solicitud GET a una URL específica
            para la provincia, la localidad y la calle. Si la respuesta es exitosa,
            se procesa el contenido HTML para extraer los datos de la tabla y se almacenan en las listas correspondientes."""
            if c.replace("Calle ","") not in [str(i) for i in range(100)]:
                ca = c.replace("/","").replace("Calle ","").replace("Boulevard ","").replace("Avenida ","").replace("Ruta Provincial ","").replace("Pasaje ","").lower().replace(" ","-").replace("ñ","n")
            else:
                ca = c.replace("/","").replace("Boulevard ","").replace("Avenida ","").replace("Ruta Provincial ","").replace("Pasaje ","").lower().replace(" ","-").replace("ñ","n")
            
            
            response2 = session.get(f'https://codigo-postal.co/argentina/{pro}/{loc}/{ca}')
            if response2.status_code == 200:
                content = response2.content
                soup = BeautifulSoup(content, "html.parser")
            
        
            calle = []
            desde = []
            hasta = []
            aplica_a = []
            cpa = []
            
            try:
                lista = soup.find("table").find("tbody").find_all("tr")
            except:
                continue
            for row in lista:
                row = row.find_all("td")
                calle.append(row[0].get_text())
                desde.append(row[1].get_text())
                hasta.append(row[2].get_text())
                aplica_a.append(row[3].get_text())
                cpa.append(row[5].get_text())
            
            street.extend(calle)
            froms.extend(desde)
            until.extend(hasta)
            isOdd.extend(aplica_a)
            zips.extend(cpa)
            localidadss = [lo] * len(calle)
            localidads.extend(localidadss)
            
    """Finalmente, se abre un nuevo archivo CSV llamado "numbers_{pro}.csv" en modo de escritura. Se especifican los nombres
    de campo para el encabezado del archivo CSV. Se utiliza csv.DictWriter para escribir las filas en el archivo CSV utilizando los valores de las listas street,
    froms, until, isOdd, zips y localidads en combinación. Cada fila representa un cpa de calle con información asociada."""
    with open(f'data/numbers_{pro}.csv', 'w', newline='') as file:
        fieldnames = ['street','froms','until','isOdd','zips','localidads']
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        
        for k in list(zip(street,froms,until,isOdd,zips,localidads)):
            
            writer.writerow({'street':k[0],'froms':k[1],'until':k[2],'isOdd':k[3],'zips':k[4],'localidads':k[5]})

    return

### Funciones de limpieza.

Limpieza y transformacion de __localities_tucuman.csv__ del ejemplo.

In [7]:
def localities_transformacion(pro):

    datos = []
    with open(f"data/localities_{pro}.csv", 'r',encoding="latin-1") as archivo:
        reader = csv.reader(archivo)
        datos = list(reader)

    # Definir los encabezados
    encabezados = ["name","zip","sate"]

    # Escribir los datos junto con los encabezados en un nuevo archivo CSV
    with open(f"localities_{pro}.csv", 'w', newline='') as archivo:
        writer = csv.writer(archivo)
        writer.writerow(encabezados)  # Escribir los encabezados en la primera fila

        for fila in datos:
            writer.writerow(fila)  # Escribir cada fila de datos

    datos = []
    with open(f"localities_{pro}.csv", 'r') as archivo:
        reader = csv.DictReader(archivo)
        for fila in reader:
            datos.append(fila)

    # Agregar la columna de ID a cada diccionario en la lista
    for i, fila in enumerate(datos):
        fila['id'] = "{:07d}".format(i+1)

    # Escribir los datos actualizados en un nuevo archivo CSV que incluya la columna de id
    encabezados = ['id'] + reader.fieldnames  # Agregar 'id' al inicio de los encabezados

    with open(f"data/localities_{pro}_final.csv", 'w', newline='') as archivo:
        writer = csv.DictWriter(archivo, fieldnames=encabezados)
        writer.writeheader()  # Escribir los encabezados en la primera fila

        for fila in datos:
            writer.writerow(fila)  # Escribir cada fila con los datos actualizados

In [8]:
localities_transformacion("tucuman")

Limpieza y transformacion de __streets_tucuman.csv__ del ejemplo.

In [15]:
def streets_transformacion(pro):

    localidad = []
    id = []
    with open(f"data/localities_{pro}_final.csv", 'r') as file:
        csv_reader = csv.reader(file)
        for i in csv_reader:
            if i[2] == "Buscar CPA":
                id.append(i[0])
                localidad.append(i[1])

    localities = dict(zip(localidad,id))

    loc_streets = []
    calle_streets = []
    with open(f"data/streets_{pro}.csv", 'r',encoding="latin-1") as file:
        csv_reader = csv.reader(file)
        for i in csv_reader:
                calle_streets.append(i[1])
                loc_streets.append(i[0])

    loc_streets_new = []
    for i in loc_streets:
        i = localities[i]
        loc_streets_new.append(i)

    types = []
    for i in calle_streets:
        for keyword in ["Calle", "Boulevard", "Avenida", "Ruta Provincial", "Pasaje"]:
            if keyword in i:    
                types.append(keyword)
                break

    with open(f'streets_{pro}_final.csv', 'w', newline='') as file:
        fieldnames = ["type","name","localityId"]
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        for k in list(zip(types,calle_streets,loc_streets_new)):
            
            writer.writerow({'type':k[0],'name':k[1],'localityId':k[2]})

    datos = []
    with open(f"streets_{pro}_final.csv", 'r') as archivo:
            reader = csv.DictReader(archivo)
            for fila in reader:
                datos.append(fila)

    # Agregar la columna de ID a cada diccionario en la lista
    for i, fila in enumerate(datos):
        fila['id'] = "{:07d}".format(i+1)

    # Escribir los datos actualizados en un nuevo archivo CSV que incluya la columna de id
    encabezados = ['id'] + reader.fieldnames  # Agregar 'id' al inicio de los encabezados

    with open(f"data/streets_{pro}_final.csv", 'w', newline='') as archivo:
        writer = csv.DictWriter(archivo, fieldnames=encabezados)
        writer.writeheader()  # Escribir los encabezados en la primera fila

        for fila in datos:
            writer.writerow(fila)  # Escribir cada fila con los datos actualizados

    return

In [16]:
streets_transformacion("tucuman")

Limpieza y transformacion de __numbers_tucuman.csv__ del ejemplo.

In [28]:
def numbers_transformacion(pro):

    par_impar = []
    calle = []
    froms = []
    untils = []
    zips = []
    with open(f"data/numbers_tucuman.csv", 'r',encoding="latin-1") as file:
        csv_reader = csv.reader(file)
        for i in csv_reader:
            
                par_impar.append(i[3])
                calle.append(i[0])
                froms.append(i[1])
                untils.append(i[2])
                zips.append(i[4])

    pa_im = {"números impares":"true","números pares":"false"}
    is0dd = []
    for i in par_impar:
        i = pa_im[i]
        is0dd.append(i)

    calless = []
    ids = []
    with open(f"data/streets_tucuman_final.csv", 'r') as file:
        csv_reader = csv.reader(file)
        for i in csv_reader:
            
                ids.append(i[0])
                calless.append(i[2])

    calles = dict(zip(calless,ids))
    
    streetId = []
    for i in calle:
            try:
                i = calles[i]
                streetId.append(i)
            except:
                 continue

    with open(f'data/numbers_tucuman_final.csv', 'w', newline='') as file:
        fieldnames = ["streetId","is0dd","from","until","zip"]
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        for k in list(zip(streetId,is0dd,froms,untils,zips)):
            
            writer.writerow({'streetId':k[0],'is0dd':k[1],'from':k[2],'until':k[3],'zip':k[4]})
    
    

    return
    

In [29]:
numbers_transformacion("tucuman")